# **Challenge 1**

Implement a **batched arbitrarily-size matrix multiplication** kernel.

Let dimensions be identical for all matrix multiplications in the batch.<br>
These being $m, k, n \in \mathbb{N}$.<br>
While $batch \in \mathbb{N}$ is the batch size.

You are provided as input the following matrices:<br>
$N_0, N_1, N_2, ... N_{batch - 1} \in \mathbb{M}^{k \times n}$<br>
$M \in \mathbb{M}^{m \times k}$

You need to compute:<br>
$P_0, P_1, P_2, ... P_{batch - 1} \in \mathbb{M}^{m \times n}$

Where $P_i = M \otimes N_i$ for each $i \in \{0, ..., batch - 1\}$.

---

A baseline reference implementation is given. Implement your version to replace it. Rely on the provided host-side function to check the correctness of results.

To get a general performance metric rely on the profiler, specifically look for the `cuda_gpu_kern_sum` and try to minimize the `Total Time (ns)` of your kernel. Meanwhile, you may also want to improve `cuda_gpu_mem_time_sum`.

Step one is beating the reference implementation, that should be easy, then you can use all tricks in the book to push it further.
Anything goes, but if you use "exotic" tricks we want an explanation.
In fact, submitting your work, be sure to fill out the [report](#report) with brief insights of what you did.

General rules and advice:
- groups are of 3 members at most, that should, for as much as possible, equally contribute to the project.
- you automagically get ~3 points in the 1st part of the exam, taking the form of 1-2 questions that you will be allowed to skip.
- deadline is 1 week (as of this writing, you will need to submit your work before october 23 at 23:59).
- submissions are to be made on WeBeep, where you need to upload a downloaded copy of this notebook (.ipynb file); for groups of multiple people, it's enough for one member to submit the file in the assignment, other members shall simply write their group's name in their submission, we will then infer groups from what you write in the report section.
- your code needs to work here on Colab with the T4 runtime.
- do not alter code sections delimited by "==="s in the final submission.
- we will change around matrix sizes arbitrarily while evaluating your work, so make sure to cover all edge cases and take care that your code is scalable (e.g. execution time grows as expected when doubling all dimensions).
- you can get the maximum grade just by using what was discussed during lectures or is present in the glossary shown during exercise sessions; still, if you wanna have "more fun" this guide is your best friend https://docs.nvidia.com/cuda/cuda-c-best-practices-guide.
- a piece of code that works is better than a supposedly faster piece of code that doesn't, so don't go overboard, but be ambitious.
- use LLMs (ChatGPT and friends) responsibly; the purpose of this challenge is for you to get your hands dirty and build up confidence in writing parallel code through trial and error. Having an LLM write your code may get you the challenge's points (unless it's so blatant that we notice), but won't lead you to learn anything and the next time you see some parallel code your mind goes blank. If you wack your head at the problem instead, and solve it, the solution will stick in the back of your mind for a long time. Similarly, if despite pushing yourself you can't find "that damn bug", then asking an LLM is fine, so long as you tried first by yourself and just say "ahhhhhh, so that what it was!" upon having the LLM help you out. Long story short, AI is fine so long as it's a tool you **learn from** and **not** one you **blindly lean on**.

If you need help or anything, please drop us an email:
- Dr. M. Ronzani: marco.ronzani@polimi.it
- Prof. F. Ferrandi: fabrizio.ferrandi@polimi.it

## **Colab Setup**

In [ ]:
%%capture
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

In [ ]:
!mkdir /home/cuda
%cd /home/cuda

## **Code**

In [ ]:
%%writefile bmatmul.cpp
// DON'T CHANGE THIS ^^ FILENAME!
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// utility for wrapping CUDA API calls and log any error they may return (use this for debugging)
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
    if (abort)
      exit(code);
  }
}

// === DO NOT CHANGE THIS ===
// host-side version, used to validate results
__host__
void batchedMatMulHost(float* M, float* N, float* P, int m, int k, int n, int batch) {
  for (int b = 0; b < batch; b++) {
    for (int row = 0; row < m; row++) {
      for (int col = 0; col < n; col++) {
        float value = 0.0f;
        for (int i = 0; i < k; i++) {
          float a = M[row*k + i];
          float c = N[b*(k*n) + i*n + col];
          value += a * c;
        }
        P[b*(m*n) + row*n + col] = value;
      }
    }
  }
}

void initWith(float number, float* arr, int size) {
  for (int i = 0; i < size; i++)
    arr[i] = number;
}

void initRandom(float* arr, int size, unsigned int seed, float minVal = 0.0f, float maxVal = 1.0f) {
  srand(seed);
  for (int i = 0; i < size; i++) {
    float r = (float)rand() / RAND_MAX;
    arr[i] = minVal + r * (maxVal - minVal);
  }
}

void checkResult(float* arr1, float* arr2, int size) {
  const float atol = 1e-4f; // absolute tolerance for fp32 (lack of) associativity
  const float rtol = 1e-4f; // relative tolerance for fp32 (lack of) associativity
  for (int i = 0; i < size; i++) {
    float diff = fabs(arr1[i] - arr2[i]);
    float tol = atol + rtol*fabs(arr2[i]);
    if (diff > tol) {
      printf("Error at %d: %f != %f (diff=%e, tol=%e)\n", i, arr1[i], arr2[i], diff, tol);
      exit(1);
    }
  }
}
// ==========================

// this is the reference implementation
// you can change this to your heart's contempt
#define TILE_SIZE 16
#define THREAD_TILE_M 4
#define THREAD_TILE_N 4
__global__
void batchedMatMul(float* M, float* N, float* P, int m, int k, int n, int batch) {
  __shared__ float tileM[TILE_SIZE * THREAD_TILE_M][TILE_SIZE];
  __shared__ float tileN[TILE_SIZE][TILE_SIZE * THREAD_TILE_N];
  int tx = threadIdx.x;
  int ty = threadIdx.y;
  int b = blockIdx.z;

  int baseRow = (blockIdx.y * TILE_SIZE + ty) * THREAD_TILE_M;
  int baseCol = (blockIdx.x * TILE_SIZE + tx) * THREAD_TILE_N;
  int numTiles = (k + TILE_SIZE - 1) / TILE_SIZE;

  float values[THREAD_TILE_M][THREAD_TILE_N] = {0.0f};

  for (int t = 0; t < numTiles; t++) {
    int kOffset = t * TILE_SIZE;

    for (int i = 0; i < THREAD_TILE_M; i++) {
      int row = baseRow + i;
      int col = kOffset + tx;
      tileM[ty * THREAD_TILE_M + i][tx] = (row < m && col < k) ? M[row * k + col] : 0.0f;
    }

    for (int j = 0; j < THREAD_TILE_N; j++) {
      int row = kOffset + ty;
      int col = baseCol + j;
      tileN[ty][tx * THREAD_TILE_N + j] = (row < k && col < n) ? N[b * (k * n) + row * n + col] : 0.0f;
    }

    __syncthreads();

    for (int kk = 0; kk < TILE_SIZE; kk++) {
      for (int i = 0; i < THREAD_TILE_M; i++) {
        float a = tileM[ty * THREAD_TILE_M + i][kk];
        for (int j = 0; j < THREAD_TILE_N; j++) {
          float b = tileN[kk][tx * THREAD_TILE_N + j];
          values[i][j] += a * b;
        }
      }
    }
    __syncthreads();
  }

  for (int i = 0; i < THREAD_TILE_M; i++) {
    for (int j = 0; j < THREAD_TILE_N; j++) {
      int row = baseRow + i;
      int col = baseCol + j;
      if (row < m && col < n) {
        P[b * (m * n) + row * n + col] = values[i][j];
      }
    }
  }
}


int main(int argc, char** argv) {
  // === DO NOT CHANGE THIS ===
  if (argc != 6) {
    printf("Usage: %s <m> <k> <n> <batch> <seed>\n", argv[0]);
    exit(1);
  }

  int m = atoi(argv[1]); // rows of Ms and Ps
  int k = atoi(argv[2]); // cols of Ms, rows of Ns
  int n = atoi(argv[3]); // cols of Ns and Ps
  int batch = atoi(argv[4]); // number of matrix pairs
  unsigned int seed = (unsigned int)atoi(argv[5]); // seed for random initialization

  printf("Running batched matmul with m=%d, k=%d, n=%d, batch=%d, seed=%u\n", m, k, n, batch, seed);

  const int sizeM = m*k;
  const int sizeN = k*n*batch;
  const int sizeP = m*n*batch;

  float* M = (float*)malloc(sizeM * sizeof(float));
  float* N = (float*)malloc(sizeN * sizeof(float));
  float* P = (float*)malloc(sizeP * sizeof(float));

  initRandom(M, sizeM, seed);
  initRandom(N, sizeN, seed + 1);
  initWith(0.0f, P, sizeP);
  // ==========================

  // here, you can change anything
  float *M_d;
  float *N_d;
  float *P_d;

  gpuErrchk(cudaMalloc((void**)&M_d, (sizeM + sizeN + sizeP) * sizeof(float)));
  // gpuErrchk(cudaMalloc((void**)&N_d, sizeN * sizeof(float)));
  // gpuErrchk(cudaMalloc((void**)&P_d, sizeP * sizeof(float)));
  N_d = M_d + sizeM;
  P_d = N_d + sizeN;

  cudaStream_t stream;
  gpuErrchk(cudaStreamCreate(&stream));

  gpuErrchk(cudaMemcpyAsync(M_d, M, sizeM * sizeof(float), cudaMemcpyHostToDevice, stream));
  gpuErrchk(cudaMemcpyAsync(N_d, N, sizeN * sizeof(float), cudaMemcpyHostToDevice, stream));
  gpuErrchk(cudaMemcpyAsync(P_d, P, sizeP * sizeof(float), cudaMemcpyHostToDevice, stream));

  dim3 blockSize(TILE_SIZE, TILE_SIZE, 1);
  dim3 numBlocks(
    (n + TILE_SIZE*THREAD_TILE_M - 1) / (TILE_SIZE*THREAD_TILE_M),
    (m + TILE_SIZE*THREAD_TILE_N - 1) /(TILE_SIZE*THREAD_TILE_N),
    batch);

  batchedMatMul<<<numBlocks, blockSize>>>(M_d, N_d, P_d, m, k, n, batch);
  gpuErrchk(cudaDeviceSynchronize());

  gpuErrchk(cudaMemcpyAsync(P, P_d, sizeP * sizeof(float), cudaMemcpyDeviceToHost, stream));

  // === DO NOT CHANGE THIS ===
  // However: once you know results are correct, you can temporarily
  //          comment this out if you want to test performance on large
  //          matrices, since the evaluation on CPU can get pretty slow.
  printf("Checking results on CPU...\n");
  float* P_host = (float*)malloc(sizeP * sizeof(float));
  initWith(0.0f, P_host, sizeP);
  batchedMatMulHost(M, N, P_host, m, k, n, batch);
  checkResult(P, P_host, m*n*batch);
  printf("All results matched, success!");
  // ==========================

  // here, you can change anything, e.g. add some logging
  gpuErrchk(cudaFree(M_d));

  free(M);
  free(N);
  free(P);
  free(P_host);

  return 0;
}


## **Compile, Run, Profile**

Compile and run:

In [ ]:
!mv bmatmul.cpp bmatmul.cu
!nvcc -arch=sm_75 bmatmul.cu -o bmatmul

In [ ]:
!./bmatmul 256 512 384 16 119

Profile:

In [ ]:
!nsys profile --stats=true ./bmatmul 256 512 384 16 119

<a name="report"></a>
## **Brief Report**

**You must fill in this section!!**

Group information:
- member-1: Tommaso, Tron, 10828422
- member-2: Francesco, Bazzano, 10828115
- member-3: Giorgio, Barocco, 10842713
- cuBLASTed <br><img src="https://i.ibb.co/vCMzb0wZ/Chat-GPT-Image-23-ott-2025-13-19-35.png" alt="+0.005 points at the exam if you know the reference ^-^" width="120" border="0">



Bullet points describing what you did with a short motivation:
- We use shared-memory tiling for M and N, letting threads load contiguous blocks, reuse data to cut global transactions to the computer memory, improving effective bandwidth and arithmetic intensity per fetched element.

- The batch dimension is mapped to grid.z so independent matrices run concurrently; this preserves coalesced accesses within M and N tiles, simplifies indexing with per-batch base pointers.

- Each thread calculates a small THREAD_TILE_M x THREAD_TILE_N block of the output, accumulating in registers to maximize reuse of loaded operands. This increases efficiency and keeps the GPU cores busier during execution. We tested different values and obtained the best results with 4x4.

- We experimented with a prefetching strategy to hide memory access latency, but the performance turned out to be worse than expected. In this context, we observed a tradeoff between kernel execution time and data transfer time, and we ultimately chose to optimize the configuration to minimize the kernel's computation time.

- Allocating one large buffer and using async copies in a dedicated stream is an attempt to reduce memory overhead.

*Note: possibly less than 8 entries of ~32 words each. More isn't necessarily better if nobody will read it.*

*Note: the subject is "the main things you came up with to improve the kernel".*